In [ ]:
import flodym as fd
import flodym.export as fde
import numpy as np

# Create two dimensions:
dims = fd.DimensionSet(
    dim_list=[
        # Future Time is the "normal" time period you'd like to investigate in the rest of the MFA system
        fd.Dimension(letter="f", name="Future Time", dtype=int, items=list(range(2025, 2051))),
        # We extend this time vector backwards to allow for stock run-up
        fd.Dimension(letter="w", name="Whole Time", dtype=int, items=list(range(1990, 2051))),
        # Historic time to define the run-up period
        fd.Dimension(letter="h", name="Historic Time", dtype=int, items=list(range(1990, 2025))),
    ]
)

# Create a stock-driven DSM with prescribed stock over the "whole time" dimension
# This includes both the historic run-up period and the future period of interest
dsm_sdriv = fd.StockDrivenDSM(
    dims=dims["w",],
    lifetime_model=fd.LogNormalLifetime(
        dims=dims["w",],
        mean=10,
        std=5,
        time_letter="w",
        ),
    time_letter="w",
)

# This is the stock we want to precribe
prescribed_stock = fd.FlodymArray(
    dims=dims["f",],
    values = np.linspace(1, 2, len(dims["f"].items)),
)

# Create a run-up stock over the historic period
# The development of this stock over time depends on your problem.
# Here, we create a linear extrapolation to zero
# You could use other methods. However, if the stock has kinks
# (jumps in the slope), that will create jumps in the inflow.
# So it is worth investing in a smooth run-up function
slope = (prescribed_stock.values[1] - prescribed_stock.values[0]) / (dims["f"].items[1] - dims["f"].items[0])
diff_to_first_future_year = dims["f"].items[0] - np.array(dims["h"].items)
values = prescribed_stock.values[0] - slope * diff_to_first_future_year
values = np.maximum(values, 0)  # avoid negative stock
runup_stock = fd.FlodymArray(
    dims=dims["h",],
    values = values,
)

# copy into the stock of the DSM
dsm_sdriv.stock[{"w": dims["h"]}] = runup_stock
dsm_sdriv.stock[{"w": dims["f"]}] = prescribed_stock

dsm_sdriv.compute()

dsm_idriv = fd.InflowDrivenDSM(
    dims=dims["w",],
    lifetime_model=dsm_sdriv.lifetime_model,
    time_letter="w",
)
dsm_idriv.inflow[...] = dsm_sdriv.inflow * material_composition * eta # eta has dimension w


# The dictionary in the MFA system will look somewhat like this
flows = {
    "use => eol": fd.FlodymArray(dims=dims["f",], name="use => eol"),
    "manufacturing => use": fd.FlodymArray(dims=dims["f",], name="manufacturing => use"),
}
# Copy only the future part of the inflows and outflows to the surrounding flows
flows["use => eol"][...] = dsm_idriv.outflow[{"w": dims["f"]}]
flows["manufacturing => use"][...] = dsm_idriv.inflow[{"w": dims["f"]}]



### Plotting


In [ ]:

# Stock (whole period)
ptr = fde.PlotlyArrayPlotter(
    array=dsm_sdriv.stock,
    intra_line_dim="Whole Time",
    title="Stock (whole period)"
)
fig = ptr.plot(do_show=True)


In [ ]:

# Stock (whole period)
ptr = fde.PlotlyArrayPlotter(
    array=dsm_sdriv.inflow,
    intra_line_dim="Whole Time",
    title="Inflow (whole period)"
)
fig = ptr.plot(do_show=True)


In [ ]:

# Relevant part of inflow
ptr = fde.PlotlyArrayPlotter(
    array=flows["manufacturing => use"],
    intra_line_dim="Future Time",
    title="Adjacent flow (future period only)"
)
fig = ptr.plot(do_show=True)



If it doesn't work, extend time back further
mean lifetime + 5*std should be a good run-up time